<a href="https://colab.research.google.com/github/prisantos/portfolio_datascience/blob/master/Desafio_Final_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O projeto consiste em uma conciliação do valor total de vendas de recarga de celular por operadora e por loja. A Bemol recebe um arquivo de uma empresa financeira (detalhamento_m4u.xlsx) onde possui o valor consolidado de vendas e o objetivo é verificar se os dados do SAP (mcg3.csv) estão de acordo com o informado. Gerar um alerta caso os valores estejam divergentes.

Dica:
Os dados do SAP vêm separados por cada transição de venda realizada. A ideia é de agrupar esses dados para deixá-los no formato parecido com que recebemos da empresa externa e, assim, poder unificar as tabelas e fazer a comparação.


# 1. Leitura dos dados 

Vamos fazer a leitura dos dados da empresa financeira (detalhamento_m4u.xlsx) e os dados do sistema SAP (mcg3.csv)

In [0]:
import pandas as pd 
import numpy as np

In [0]:
df_venda = pd.read_csv('/content/mcg3.csv', sep='|', encoding='utf8')

In [0]:
df_financeira = pd.concat(pd.read_excel('/content/detalhamento_m4u.xlsx',sheet_name=None, skiprows=1), ignore_index=True)

# 2. Tratamento de Dados  

2.1 Verificando os DataFrames

In [0]:
df_venda.head()


,Escritório de vendas,Fornecedor,Material,Data,Val.líq.
0,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",02.01.2020,20.0
1,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",30.12.2019,10.0
2,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",04.01.2020,20.0
3,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",02.01.2020,10.0
4,101,VIVO,"131318 RECARGA VIRTUAL VIVO R$ 1,00",03.01.2020,25.0


2.3 Renomeando colunas 

Vamos renomear as colunas para melhorar a compreensão e para a operação de merge

In [0]:
df_venda = df_venda.rename(columns = {'Escritório de vendas':'local_vendas', 'Fornecedor':'fornecedor','Val.líq.':'Valor_liq'})

2.4 Verificação dos tipos

In [0]:
df_venda.dtypes

local_vendas      int64
fornecedor       object
Material         object
Data             object
Valor_liq       float64
dtype: object

2.6 Deletar dados 

Vamos excluir a coluna Material que não iremos utilizar 

In [0]:
df_venda = df_venda.drop(['Material', 'Data'], axis='columns')

Vamos verificar  os tipos do dataframe Venda

In [0]:
df_venda.dtypes


local_vendas      int64
fornecedor       object
Valor_liq       float64
dtype: object

# 3 Concatenar Tabelas

In [0]:
df_financeira.head()

,NomeFantasia,Escritório de vendas,UF,Operadora,Vlookup Bruto
0,BEMOL AVENIDA,103,AM,Claro,2651
1,BEMOL AVENIDA,103,AM,Oi,3309
2,BEMOL AVENIDA,103,AM,Tim,1550
3,BEMOL AVENIDA,103,AM,Vivo,4739
4,BEMOL BARROSO,107,AM,Claro,1495


Vamos excluir a coluna NomeFantasia que não iremos utilizar 

In [0]:
df_financeira = df_financeira.drop(['NomeFantasia'], axis='columns')

Agora vamos renomear as colunas do dataframe df_financeira

In [0]:
df_financeira = df_financeira.rename(columns = {'Operadora':'fornecedor','Escritório de vendas':'local_vendas', 'Vlookup Bruto': 'Valor_bruto'})

Como iremos comparar nome de fornecedor dos dois dataframes, iremos alterar o formato para maiúsculas 

In [0]:
df_financeira['fornecedor'] = df_financeira['fornecedor'].str.upper()

Verificamos que no dataframe tem uma fornecedora que é a Antiga OI, estamos atualizando para depois fazer a correlação 

In [0]:
 df_financeira = df_financeira.apply(lambda x: x.replace('TNL','OI'))

In [0]:
 df_venda = df_venda.apply(lambda x: x.replace('TNL','OI'))

Vamos verificar o tipo do dataframe df_financeira

In [0]:
df_financeira.dtypes

local_vendas     int64
UF              object
fornecedor      object
Valor_bruto      int64
dtype: object

Verificamos que o tipo entre a coluna valor_bruto e valor_liquido são divergentes. Iremos atualizar o tipo para realizar a comparação corretamente.

In [0]:
df_financeira['Valor_bruto'] = df_financeira['Valor_bruto'].astype(float)

Vamos concatenar os dataframes venda e financeira 

In [0]:
df_venda = df_venda.groupby(['local_vendas', 'fornecedor']).agg({'Valor_liq':'sum'}).reset_index()


In [0]:
df_financeira = df_financeira.groupby(['local_vendas', 'fornecedor']).agg({ 'Valor_bruto':'first'}).reset_index()

# Correlação de Tabelas 

Vamos fazer a correlação dos dataframe venda e financeira

In [0]:
df_relatorio = pd.merge(df_venda, df_financeira, on = ['local_vendas', 'fornecedor'], how = 'outer')

In [0]:
df_relatorio.head (5)

,local_vendas,fornecedor,Valor_liq,Valor_bruto
0,101,CLARO,1527.0,1527.0
1,101,OI,1730.0,1730.0
2,101,TIM,975.0,975.0
3,101,VIVO,3142.0,3142.0
4,103,CLARO,2652.3,2651.0


Verificar se existem valores ausentes NAN

In [0]:
df_relatorio.isnull().sum()

local_vendas    0
fornecedor      0
Valor_liq       0
Valor_bruto     1
dtype: int64

In [0]:
df_relatorio = df_relatorio.fillna(0)

In [0]:
def comparacao (valor_liq, valor_bruto):
  if(valor_liq == valor_bruto):
    return ''
  else:
    return 'alerta'

In [0]:
df_relatorio['status'] = df_relatorio.apply(lambda row: comparacao(row['Valor_bruto'],row['Valor_liq']), axis='columns')

In [0]:
df_relatorio.head(15)

,local_vendas,fornecedor,Valor_liq,Valor_bruto,status
0,101,CLARO,1527.0,1527.0,
1,101,OI,1730.0,1730.0,
2,101,TIM,975.0,975.0,
3,101,VIVO,3142.0,3142.0,
4,103,CLARO,2652.3,2651.0,alerta
5,103,OI,3309.0,3309.0,
6,103,TIM,1550.0,1550.0,
7,103,VIVO,4739.0,4739.0,
8,105,CLARO,720.0,720.0,
9,105,OI,2030.0,2030.0,


In [0]:
df_relatorio.to_excel('Total_vendas.xlsx',encoding='utf-8',index=False)